**Features for schizophrenia classification:**


1.   **Gamma Power**: Measures the power within the gamma frequency range (30-100 Hz), as alterations in gamma oscillations are often associated with cognitive dysfunction in schizophrenia.
2.   **Relative Gamma Power**: Represents the proportion of gamma power relative to total power, highlighting the prominence of gamma activity in schizophrenia-related brain alterations.
3.   **Connectivity**: Assesses functional connectivity between brain regions, as schizophrenia often shows disrupted communication between distant brain regions.
4.   **Phase-Locking Value (PLV)**: Quantifies phase synchronization between EEG signals, capturing impairments in the coherence of brain oscillations in schizophrenia.
5.   **Coherence**: Evaluates the synchrony between different EEG channels, reflecting abnormal connectivity and network interactions in schizophrenia.





In [ ]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.0 MB/s eta 0:00:00


In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_array = np.load('/content/drive/My Drive/data_array.npy')
print(data_array.shape)

(6843, 19, 1250)


In [ ]:
from scipy.signal import welch,coherence,hilbert

In [ ]:
# Define gamma frequency range
FREQ_MIN, FREQ_MAX = 30, 100  # Gamma band
SFREQ = 250  # Sampling frequency
N_PERSEG = 512  # Segment length for Welch

def gamma_power(x):

  freqs, psd = welch(x, fs=SFREQ, nperseg=512)  # Compute PSD
  gamma_idx = np.logical_and(freqs >= FREQ_MIN, freqs <= FREQ_MAX)
  return np.mean(psd[:, gamma_idx], axis=-1)  # Compute mean gamma power

def relative_gamma_power(x):

  freqs, psd = welch(x, fs=SFREQ, nperseg=512)  # Compute PSD
  gamma_idx = np.logical_and(freqs >= FREQ_MIN, freqs <= FREQ_MAX)

  gamma_power = np.mean(psd[:, gamma_idx], axis=-1)
  total_power = np.sum(psd, axis=-1)  # Sum across all frequencies

  return gamma_power / total_power  # Relative power


# Function to compute Connectivity (Correlation)
def connectivity(x):
    # Compute correlation matrix (cross-correlation of all channel pairs)
    corr_matrix = np.corrcoef(x)  # x is of shape (n_channels, n_samples)

    # Flatten the upper triangular part (excluding diagonal) for connectivity features
    # This gives a symmetric matrix, so we don't need both sides
    upper_triangular = corr_matrix[np.triu_indices(corr_matrix.shape[0], 1)]

    return upper_triangular

# Function to compute Phase-Locking Value (PLV)
def plv(x):
    analytic_signal = hilbert(x)  # Apply Hilbert Transform
    phases = np.angle(analytic_signal)  # Extract phase
    plv_matrix = np.zeros((x.shape[0], x.shape[0]))  # (19, 19)
    for i in range(x.shape[0]):
        for j in range(i + 1, x.shape[0]):
            plv_matrix[i, j] = np.abs(np.mean(np.exp(1j * (phases[i] - phases[j]))))
    return plv_matrix[np.triu_indices(x.shape[0], k=1)]  # Upper triangle values (171,)


# Function to compute Coherence between channel pairs
def coh(x):
    coh_matrix = np.zeros((x.shape[0], x.shape[0]))  # (19, 19)
    # Compute coherence for all pairs in a vectorized manner
    freqs, _ = coherence(x[0], x[0], fs=SFREQ, nperseg=512)  # Get frequency bins once
    gamma_idx = np.logical_and(freqs >= FREQ_MIN, freqs <= FREQ_MAX)  # Precompute gamma band index

    for i in range(x.shape[0]):
        for j in range(i + 1, x.shape[0]):
            _, Cxy  = coherence(x[i], x[j], fs=SFREQ, nperseg=512)
            coh_matrix[i, j] = np.mean(Cxy[gamma_idx])  # Extract gamma band coherence
            coh_matrix[j, i] = coh_matrix[i, j]  # Fill the symmetric matrix

    return coh_matrix[np.triu_indices(x.shape[0], k=1)]  # Upper triangle values (171,)


# Function to concatenate features
def concatenate_features(x):
    return np.concatenate((gamma_power(x),relative_gamma_power(x),connectivity(x),plv(x),coh(x)), axis=-1)


In [ ]:
features=[]
for d in data_array:
  features.append(concatenate_features(d))

In [ ]:
features_array = np.array(features)
features_array.shape

(6843, 551)

In [ ]:
# Save the array
np.save('/content/drive/My Drive/features_array.npy', features_array)